# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple, Inc.',
 'primaryExchange': 'AQANDS',
 'calculationPrice': 'tops',
 'open': None,
 'openTime': None,
 'openSource': 'aliifofc',
 'close': None,
 'closeTime': None,
 'closeSource': 'aoiifclf',
 'high': 464.8,
 'highTime': 1656967034950,
 'highSource': 'teerapiee imd1c dun l5y',
 'low': 456.62,
 'lowTime': 1608606402397,
 'lowSource': '1eymrldetu cidenepi 5 a',
 'latestPrice': 466.85,
 'latestSource': 'IEX real time price',
 'latestTime': '3:54:14 PM',
 'latestUpdate': 1645743524276,
 'latestVolume': 49245227,
 'iexRealtimePrice': 461.33,
 'iexRealtimeSize': 104,
 'iexLastUpdated': 1623856638761,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 472.78,
 'previousVolume': 42365885,
 'change': 8.72,
 'changePercent': 0.01906,
 'volume': 49156172,
 'iexMarketPerc

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

35.81

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,102.460,45.7,N/A
1,AAL,13.660,-1.63,N/A
2,AAP,159.980,28.5,N/A
3,AAPL,475.730,36.14,N/A
4,ABBV,96.910,20.89,N/A
...,...,...,...,...
500,YUM,94.260,28.35,N/A
501,ZBH,143.350,734.27,N/A
502,ZBRA,296.400,32.26,N/A
503,ZION,35.900,13.5,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,RTX,65.160,2454.9,N/A
1,FIS,145.640,1532.03,N/A
2,GPC,97.298,937.44,N/A
3,ZBH,143.350,734.27,N/A
4,STZ,177.130,595.23,N/A
5,HLT,89.363,588.57,N/A
6,HRB,15.331,542,N/A
7,GLW,33.361,250.4,N/A
8,IQV,168.780,243.7,N/A
9,WMB,22.070,213.06,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,RTX,65.160,2454.9,300
1,FIS,145.640,1532.03,134
2,GPC,97.298,937.44,201
3,ZBH,143.350,734.27,136
4,STZ,177.130,595.23,110
5,HLT,89.363,588.57,219
6,HRB,15.331,542,1278
7,GLW,33.361,250.4,587
8,IQV,168.780,243.7,116
9,WMB,22.070,213.06,888


## Building a Better (and More Realistic) Momentum Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'Price-to-Earnings Ratio',
                'PE Percentile',
                'Price-to-Book Ratio',
                'PB Percentile',
                'EV/EBITDA',
                'EV/EBITDA Percentile',
                'EV/GP',
                'EV/GP Percentile',
                'RV Score'
                ]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        
        pe_ratio = data[symbol]['quote']['peRatio']
        pb_ratio = data[symbol]['advanced-stats']['priceToBook']
        ps_ratio = data[symbol]['advanced-stats']['priceToSales']
        
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
                                        pd.Series([
                                                    symbol, 
                                                    data[symbol]['quote']['latestPrice'], 
                                                    'N/A', 
                                                    pe_ratio,
                                                    'N/A',
                                                    pb_ratio,
                                                    'N/A',
                                                    ev_to_ebitda,
                                                    'N/A',
                                                    ev_to_gross_profit,
                                                    'N/A',
                                                    'N/A'
                                                   ], 
                                                  index = rv_columns), 
                                        ignore_index = True)
        

rv_dataframe.set_index('Ticker', inplace = True)
rv_dataframe

,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
Ticker,,,,,,,,,,,
A,100.800,N/A,46.11,N/A,6.584435,N/A,27.633528,N/A,11.278899,N/A,N/A
AAL,13.410,N/A,-1.64,N/A,-61.217457,N/A,6.194679,N/A,3.136683,N/A,N/A
AAP,161.210,N/A,28.84,N/A,3.075461,N/A,14.317298,N/A,3.086607,N/A,N/A
AAPL,469.715,N/A,35.35,N/A,21.949982,N/A,26.506599,N/A,20.390743,N/A,N/A
ABBV,97.620,N/A,20.62,N/A,-20.861183,N/A,12.412566,N/A,7.646713,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...
YUM,96.070,N/A,27.59,N/A,-3.597017,N/A,18.573913,N/A,13.825263,N/A,N/A
ZBH,144.300,N/A,732.91,N/A,2.371745,N/A,17.114624,N/A,7.465194,N/A,N/A
ZBRA,292.510,N/A,31.73,N/A,8.650796,N/A,19.140447,N/A,8.342880,N/A,N/A


In [13]:
print(data[symbol]['advanced-stats'])

{'week52change': 0.281491, 'week52high': 164.04, 'week52low': 93.48, 'marketcap': 77126198907, 'employees': 10954, 'day200MovingAvg': 135.29, 'day50MovingAvg': 142.88, 'float': 486360753, 'avg10Volume': 1649885.3, 'avg30Volume': 1470768.26, 'ttmEPS': 3.5127, 'ttmDividendRate': 0.798, 'companyName': 'Zoetis, Inc.', 'sharesOutstanding': 481288954, 'maxChangePercent': 3.4018, 'year5ChangePercent': 1.684, 'year2ChangePercent': 0.6011, 'year1ChangePercent': 0.282739, 'ytdChangePercent': 0.18994, 'month6ChangePercent': 0.099747, 'month3ChangePercent': 0.298343, 'month1ChangePercent': 0.153119, 'day30ChangePercent': 0.163893, 'day5ChangePercent': -0.012759, 'nextDividendDate': None, 'dividendYield': 0.00482771109557038, 'nextEarningsDate': '2020-10-16', 'exDividendDate': '2020-07-03', 'peRatio': 48.39, 'beta': 0.9755668184449848, 'totalCash': 1969326458, 'currentDebt': 558463722, 'revenue': 6370126818, 'grossProfit': 4255781949, 'totalRevenue': 6555364956, 'EBITDA': 2535545585, 'revenuePerSha

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
Ticker,,,,,,,,,,,
AFL,38.541,N/A,9.61,N/A,0.939471,N/A,NaN,N/A,NaN,N/A,N/A
AIG,32.112,N/A,-5.71,N/A,0.404919,N/A,NaN,N/A,NaN,N/A,N/A
AIZ,126.220,N/A,20.14,N/A,1.359035,N/A,NaN,N/A,NaN,N/A,N/A
ALL,99.893,N/A,7.09,N/A,1.174081,N/A,NaN,N/A,NaN,N/A,N/A
ANTM,284.030,N/A,12.41,N/A,2.269777,N/A,NaN,N/A,NaN,N/A,N/A
AON,201.360,N/A,26.28,N/A,13.733424,N/A,17.945864,N/A,NaN,N/A,N/A
BAC,27.101,N/A,13.1,N/A,0.870710,N/A,NaN,N/A,NaN,N/A,N/A
BK,38.344,N/A,8.18,N/A,0.794885,N/A,NaN,N/A,NaN,N/A,N/A
BKNG,1858.780,N/A,31.21,N/A,12.910286,N/A,12.790478,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
for column in ['Price-to-Earnings Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
Ticker,,,,,,,,,,,


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [18]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

Ticker
A       0.841584
AAL     0.114851
AAP     0.621782
AAPL    0.738614
ABBV    0.421782
          ...   
YUM     0.592079
ZBH     0.994059
ZBRA     0.69901
ZION    0.267327
ZTS     0.843564
Name: PE Percentile, Length: 505, dtype: object
Ticker
A        0.746535
AAL     0.0158416
AAP       0.50495
AAPL     0.936634
ABBV    0.0257426
          ...    
YUM      0.049505
ZBH      0.411881
ZBRA     0.815842
ZION     0.134653
ZTS      0.954455
Name: PB Percentile, Length: 505, dtype: object
Ticker
A        0.893069
AAL     0.0851485
AAP      0.455446
AAPL     0.879208
ABBV     0.346535
          ...    
YUM      0.625743
ZBH      0.576238
ZBRA     0.750495
ZION     0.679208
ZTS      0.922772
Name: EV/EBITDA Percentile, Length: 505, dtype: object
Ticker
A        0.548515
AAL     0.0594059
AAP     0.0554455
AAPL     0.879208
ABBV     0.340594
          ...    
YUM      0.740594
ZBH      0.324752
ZBRA     0.382178
ZION     0.644554
ZTS      0.853465
Name: EV/GP Percentile, Length: 505, dty

,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
Ticker,,,,,,,,,,,
A,100.800,N/A,46.11,0.841584,6.584435,0.746535,27.633528,0.893069,11.278899,0.548515,N/A
AAL,13.410,N/A,-1.64,0.114851,-61.217457,0.0158416,6.194679,0.0851485,3.136683,0.0594059,N/A
AAP,161.210,N/A,28.84,0.621782,3.075461,0.50495,14.317298,0.455446,3.086607,0.0554455,N/A
AAPL,469.715,N/A,35.35,0.738614,21.949982,0.936634,26.506599,0.879208,20.390743,0.879208,N/A
ABBV,97.620,N/A,20.62,0.421782,-20.861183,0.0257426,12.412566,0.346535,7.646713,0.340594,N/A
...,...,...,...,...,...,...,...,...,...,...,...
YUM,96.070,N/A,27.59,0.592079,-3.597017,0.049505,18.573913,0.625743,13.825263,0.740594,N/A
ZBH,144.300,N/A,732.91,0.994059,2.371745,0.411881,17.114624,0.576238,7.465194,0.324752,N/A
ZBRA,292.510,N/A,31.73,0.69901,8.650796,0.815842,19.140447,0.750495,8.342880,0.382178,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [19]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
Ticker,,,,,,,,,,,
A,100.800,N/A,46.11,0.841584,6.584435,0.746535,27.633528,0.893069,11.278899,0.548515,0.757426
AAL,13.410,N/A,-1.64,0.114851,-61.217457,0.0158416,6.194679,0.0851485,3.136683,0.0594059,0.0688119
AAP,161.210,N/A,28.84,0.621782,3.075461,0.50495,14.317298,0.455446,3.086607,0.0554455,0.409406
AAPL,469.715,N/A,35.35,0.738614,21.949982,0.936634,26.506599,0.879208,20.390743,0.879208,0.858416
ABBV,97.620,N/A,20.62,0.421782,-20.861183,0.0257426,12.412566,0.346535,7.646713,0.340594,0.283663
...,...,...,...,...,...,...,...,...,...,...,...
YUM,96.070,N/A,27.59,0.592079,-3.597017,0.049505,18.573913,0.625743,13.825263,0.740594,0.50198
ZBH,144.300,N/A,732.91,0.994059,2.371745,0.411881,17.114624,0.576238,7.465194,0.324752,0.576733
ZBRA,292.510,N/A,31.73,0.69901,8.650796,0.815842,19.140447,0.750495,8.342880,0.382178,0.661881
